In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import sqlalchemy

In [3]:
engine = sqlalchemy.create_engine(os.getenv("DATABASE_URL"))

In [4]:
!pwd

/home/primefactorx01/lambda/labs/labspt15-cityspire-g-ds


In [6]:
df = pd.read_csv('datasets_to_merge/four_merged.csv')

In [8]:
df.to_sql("data", con=engine)

In [9]:
pd.read_sql('data', con=engine)

,index,City,State,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Level of Concern
0,0,Akron,OH,267774,129620,138083,1.9,64.2,26.6,0.1,...,107,63,43,0,0,132,0,142,0,Good
1,1,Albany,GA,93421,43571,49835,3.2,36.6,56.8,0.1,...,114,63,37,0,0,0,0,244,0,Good
2,2,Albany,NY,147794,70664,77096,7.1,58.3,21.9,0.1,...,75,53,39,1,0,169,0,104,0,Good
3,3,Albany,OR,61555,30378,31168,10.4,81.7,0.3,0.6,...,70,51,19,0,0,0,0,182,0,Good
4,4,Albuquerque,NM,632237,309415,322758,46.8,41.0,2.1,4.8,...,100,65,50,1,0,127,0,25,30,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,371,Worcester,MA,183628,89977,93611,21.5,57.6,10.7,0.2,...,90,52,40,0,6,169,0,61,1,Good
372,372,Wyoming,MI,56377,27516,28855,26.3,57.4,6.9,0.3,...,150,58,38,2,0,131,0,50,0,Good
373,373,Yakima,WA,125035,61639,63374,35.5,57.6,0.7,2.3,...,90,54,27,0,0,0,0,243,1,Good
374,374,Yuba City,CA,75848,37647,38198,26.4,50.5,1.9,0.5,...,159,61,43,0,0,149,0,63,1,Good


In [10]:
df.columns

Index(['City', 'State', 'TotalPop', 'Men', 'Women', 'Hispanic', 'White',
       'Black', 'Native', 'Asian', 'Pacific', 'Income', 'IncomeErr',
       'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty',
       'Employed', 'Unemployment', 'PrivateWork', 'PublicWork', 'SelfEmployed',
       'FamilyWork', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Unnamed: 0_x', 'Population',
       'Violent crime', 'Murder and nonnegligent manslaughter', 'Rape',
       'Robbery', 'Aggravated assault', 'Property crime', 'Burglary',
       'Larceny- theft', 'Motor vehicle theft', 'Arson', 'Crime Percentage',
       'Crime Rating', 'Unnamed: 0_y', 'Rent', 'Year', 'Days with AQI',
       'Good Days', 'Moderate Days', 'Unhealthy for Sensitive Groups Days',
       'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th Percentile AQI', 'Median AQI', 'Days 

In [15]:
df['Level of Concern'].value_counts()

Good        353
Moderate     23
Name: Level of Concern, dtype: int64

In [18]:
lat_lon = pd.read_csv('notebooks/cities100pop.csv', index_col='Unnamed: 0')

In [21]:
lat_lon.drop(['State', 'SizeRank', '2020-12'], 1)

,City,lat,lon
0,United States,39.6467,-100.6623
1,New York,40.8154,-73.9296
2,Anaheim,33.8303,-117.9128
3,Long Beach,33.7824,-118.1967
4,Los Angeles,34.0059,-118.2853
...,...,...,...
106,Port St. Lucie,27.2862,-80.3760
107,Fort Collins,40.5534,105.0869
108,Boulder,40.0271,105.2572
109,Greeley,40.4067,-104.7764


In [22]:
import geopy

In [26]:
geolocater=geopy.Nominatim(user_agent='cityspire-g')

In [27]:
geolocater .geocode('Sacramento, CA')

Location(Sacramento, Sacramento County, California, United States, (38.5810606, -121.4938951, 0.0))

In [41]:
lat_lon = [geolocater.geocode(
    f"{row[0]}, {row[1]}") 
for row in zip(df['City'], df['State'])]

In [50]:
df_lat = pd.DataFrame({'lat':[r[1][0] for r in lat_lon], 'lon': [r[1][1] for r in lat_lon]})

In [52]:
df2 = df.copy()

In [53]:
df2[['lat', 'lon']] = df_lat

In [55]:
df2.to_sql("data", con=engine, if_exists = 'replace')

In [33]:
df['City']

0            Akron
1           Albany
2           Albany
3           Albany
4      Albuquerque
          ...     
371      Worcester
372        Wyoming
373         Yakima
374      Yuba City
375           Yuma
Name: City, Length: 376, dtype: object

In [57]:
df2.to_csv('datasets_to_merge/four_merged.csv')